## 1.1. Продвинутые концепции Python

### 1.1.1. Декораторы и их применение

**Декоратор** - это функция, которая принимает другую функцию в качестве аргумента, добавляет к ней некоторую функциональность и возвращает новую функцию

Создание простого декоратора

In [5]:
# декоратор, который будет выводить сообщение до и после вызова функции.
def logger(func):
    def wrapper(*args, **kwargs):
        print(f"Вызов функции {func.__name__} с аргументами {args} и {kwargs}")
        result = func(*args, **kwargs)
        print(f"Функция {func.__name__} завершила выполнение")
        return result
    return wrapper

@logger
def add(a, b):
    return a + b

print(add(2, 3))

Вызов функции add с аргументами (2, 3) и {}
Функция add завершила выполнение
5


Создание декоратора с аргументами

In [7]:
# декоратор, который повторяет выполнение функции заданное количество раз
def repeat(num_times):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator

@repeat(3)
def greet(name):
    print(f"Привет, {name}!")

greet("Alex")

Привет, Alex!
Привет, Alex!
Привет, Alex!


Встроенные декораторы. @staticmethod и @classmethod и @property  
- @staticmethod используется для создания методов, которые не требуют доступа к экземпляру или классу.
- @classmethod используется для создания методов, которые работают с классом, а не с экземпляром.
- @property позволяет превратить метод в атрибут, который можно читать без вызова.

Декораторы можно применять последовательно. Порядок применения важен: декораторы выполняются снизу вверх.

#### Практика

In [11]:
# декоратор, который измеряет время выполнения функции и выводит его
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Функция {func.__name__} выполнилась за {end_time - start_time:.4f} секунд")
        return result
    return wrapper

@timer
def slow_function(sec):
    time.sleep(sec)

slow_function(3)

Функция slow_function выполнилась за 3.0015 секунд


In [12]:
# декоратор, который кэширует результаты функции, чтобы избежать повторных вычислений
def cache(func):
    cached_results = {}
    def wrapper(*args):
        if args in cached_results:
            print("Результат из кэша")
            return cached_results[args]
        result = func(*args)
        cached_results[args] = result
        print("Результат вычислен")
        return result
    return wrapper

@cache
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print(fibonacci(2))

Результат вычислен
Результат вычислен
Результат вычислен
1


In [13]:
# Декоратор для логирования аргументов и результата функции в файл
def log_to_file(filename):
    def decorator(func):
        def wrapper(*args, **kwargs):
            with open(filename, 'a') as f:
                f.write(f"Вызов функции {func.__name__} с аргументами: {args}, {kwargs}\n")
            result = func(*args, **kwargs)
            with open(filename, 'a') as f:
                f.write(f"Результат функции {func.__name__}: {result}\n")
            return result
        return wrapper
    return decorator

@log_to_file('log.txt')
def add(a, b):
    return a + b

add(2, 3)

5

In [14]:
# Декоратор для проверки, что аргументы функции являются числами
def check_numeric(func):
    def wrapper(*args, **kwargs):
        # Проверка позиционных аргументов (args)
        for arg in args:
            if not isinstance(arg, (int, float)):
                raise ValueError(f"Аргумент {arg} не является числом")

        # Проверка именованных аргументов (kwargs)
        for key, value in kwargs.items():
            if not isinstance(value, (int, float)):
                raise ValueError(f"Аргумент {key}={value} не является числом")

        # Если все аргументы прошли проверку, вызываем исходную функцию
        return func(*args, **kwargs)
    return wrapper

@check_numeric
def multiply(a, b):
    return a * b

print(multiply(2, 3))  # Работает
print(multiply(2, '3'))  # Вызовет исключение

6


ValueError: Аргумент 3 не является числом

In [15]:
# Декоратор для ограничения количества вызовов функции
def limit_calls(max_calls):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if wrapper.calls >= max_calls:
                raise Exception(f"Превышено максимальное количество вызовов функции {func.__name__}")
            wrapper.calls += 1
            return func(*args, **kwargs)
        wrapper.calls = 0
        return wrapper
    return decorator

@limit_calls(2)
def say_hello(name):
    print("Hello", name)

say_hello("Vasay1")
say_hello("Vasay2")
say_hello("Vasay3")

Hello Vasay1
Hello Vasay2


Exception: Превышено максимальное количество вызовов функции say_hello

In [17]:
@log_to_file('log.txt')
@check_numeric
@limit_calls(3)
def add(a, b):
    return a + b

add(2, 3)
add(4, 5)
add(6, 7)
add(8, 9)  # Вызовет исключение из-за ограничения на количество вызовов

Exception: Превышено максимальное количество вызовов функции add

### 1.1.2. Генераторы и итераторы

Генераторы и итераторы — это мощные инструменты Python для работы с последовательностями данных. Они позволяют эффективно работать с большими объемами данных, не загружая их полностью в память.

Итератор - это объект, который реализует протокол итерации.  
- \_\_iter__() - возвращает сам объект итератора.
- \_\_next__() - возвращает следующий элемент последовательности. Если элементы закончились, вызывается исключение StopIteration.

In [22]:
# простой итератор, который возвращает числа от 0 до заданного предела
class MyIterator:
    def __init__(self, limit):
        self.limit = limit
        self.current = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current < self.limit:
            self.current += 1
            return self.current - 1
        else:
            raise StopIteration

my_iter = MyIterator(3)
for num in my_iter:
    print(num)

0
1
2


Генератор — это функция, которая возвращает итератор. Вместо return используется ключевое слово yield, которое приостанавливает выполнение функции и возвращает значение. При следующем вызове функция продолжает выполнение с того места, где она была приостановлена.

In [25]:
# генератор, который возвращает числа от 0 до заданного предела
def my_generator(limit):
    current = 0
    while current < limit:
        yield current
        current += 1

gen = my_generator(3)
for num in gen:
    print(num)

0
1
2


In [27]:
# генератор, который возвращает квадраты чисел.
squares = (x * x for x in range(5))

for square in squares:
    print(square, end=" ")

0 1 4 9 16 

#### Практика

In [30]:
# генератор, который возвращает числа Фибоначчи до заданного предела
def fibonacci_generator(limit):
    a, b = 0, 1
    while a < limit:
        yield a
        a, b = b, a + b

for num in fibonacci_generator(200):
    print(num, end=" ")

0 1 1 2 3 5 8 13 21 34 55 89 144 

In [32]:
# итератор, который возвращает элементы списка в обратном порядке
class ReverseIterator:
    def __init__(self, data):
        self.data = data
        self.index = len(data)

    def __iter__(self):
        return self

    def __next__(self):
        if self.index > 0:
            self.index -= 1
            return self.data[self.index]
        else:
            raise StopIteration

# Использование итератора
my_list = [1, 2, 3, 4, 5]
rev_iter = ReverseIterator(my_list)
for item in rev_iter:
    print(item)

5
4
3
2
1


In [34]:
# генератор, который возвращает бесконечную последовательность чисел, начиная с заданного числа
def infinite_sequence(start):
    current = start
    while True:
        yield current
        current += 1

# Использование генератора (осторожно: бесконечный цикл!)
gen = infinite_sequence(10)
for _ in range(5):
    print(next(gen))

10
11
12
13
14


### 1.1.3. Контекстные менеджеры (with)

### 1.1.4. Работа с исключениями и кастомные исключения

### 1.1.5. Модуль collections (defaultdict, Counter, namedtuple)

### 1.1.6. Написание утилит с использованием декораторов и генераторов